In [9]:
import findspark
findspark.find()

findspark.init("/Users/yanji/opt/spark")

import pyspark

spark = pyspark.sql.SparkSession.builder \
    .master("local") \
    .appName("ch1") \
    .config("spark.driver.bindAddress","192.168.0.164") \
    .config("spark.ui.port","4050") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

print("http://{}:{}".format(spark.conf.get("spark.driver.bindAddress"), spark.conf.get("spark.ui.port")))


http://192.168.0.164:4050


In [10]:
df = spark.read.option("inferSchema", "true").option("header", "true")\
    .csv("../data/flight-data/csv/2015-summary.csv")

In [11]:
df.sort("count").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Estonia|    1|
|              Kosovo|      United States|    1|
|              Zambia|      United States|    1|
|       United States|   Papua New Guinea|    1|
|               Malta|      United States|    1|
|       United States|          Gibraltar|    1|
|            Suriname|      United States|    1|
|       United States|            Croatia|    1|
|            Djibouti|      United States|    1|
|        Burkina Faso|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|             Cyprus|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
|              Cyprus|      United States|    1|
|       United States|          Lithuania|    1|
|       United States|           Bulgaria|    1|
|       United State

In [12]:
df.createOrReplaceTempView("flight_data_2015")
sql = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1) AS count
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME ORDER BY count DESC
""")
dft = df.groupby("DEST_COUNTRY_NAME").count()

In [13]:
maxsql = spark.sql('''
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY destination_total DESC
'''
)
# maxsql.show()
maxsql.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [destination_total#116L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(destination_total#116L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [id=#114]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#83], functions=[sum(count#85)])
         +- Exchange hashpartitioning(DEST_COUNTRY_NAME#83, 200), ENSURE_REQUIREMENTS, [id=#111]
            +- HashAggregate(keys=[DEST_COUNTRY_NAME#83], functions=[partial_sum(count#85)])
               +- FileScan csv [DEST_COUNTRY_NAME#83,count#85] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/yanji/Documents/codes/Spark-The-Definitive-Guide/data/flig..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




In [14]:
df.groupBy("DEST_COUNTRY_NAME").sum("count").withColumnRenamed("sum(count)", "destination_total") \
    .sort('destination_total', ascending=False).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [destination_total#129L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(destination_total#129L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [id=#134]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#83], functions=[sum(count#85)])
         +- Exchange hashpartitioning(DEST_COUNTRY_NAME#83, 200), ENSURE_REQUIREMENTS, [id=#131]
            +- HashAggregate(keys=[DEST_COUNTRY_NAME#83], functions=[partial_sum(count#85)])
               +- FileScan csv [DEST_COUNTRY_NAME#83,count#85] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/yanji/Documents/codes/Spark-The-Definitive-Guide/data/flig..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


